In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Read the dataset

games= pd.read_csv("C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/games.csv")
player_play=pd.read_csv("C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/player_play.csv")
players=pd.read_csv("C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/players.csv")
plays=pd.read_csv("C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/plays.csv")
track_w1=pd.read_csv("C:/Users/lanvu\Downloads/nfl-big-data-bowl-2025/tracking_week_1.csv")
track_w2=pd.read_csv('C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_2.csv')
track_w3=pd.read_csv('C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_3.csv')
track_w4=pd.read_csv('C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_4.csv')
track_w5=pd.read_csv('C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_5.csv')
track_w6=pd.read_csv('C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_6.csv')
track_w7=pd.read_csv('C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_7.csv')
track_w8=pd.read_csv('C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_8.csv')
track_w9=pd.read_csv('C:/Users/lanvu/Downloads/nfl-big-data-bowl-2025/tracking_week_9.csv')


In [ ]:
#EDA
import matplotlib.pyplot as plt
games['homeWin'] = games['homeFinalScore'] > games['visitorFinalScore']

# Count wins and losses
win_counts = games['homeWin'].value_counts()

# Prepare labels and values
labels = ['Home Wins', 'Home Losses']
sizes = [win_counts[True], win_counts[False]] if False in win_counts.index else [win_counts[True], 0]

# Plot the pie chart
plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title('Proportion of Home Wins')
plt.show()


In [ ]:
players.columns

In [ ]:
plays['pff_passCoverage'].unique()

In [ ]:
# Select relevant columns
passCoverage_analysis = plays[['pff_passCoverage','yardsGained', 'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'expectedPointsAdded']]

# Group by isShotgun and calculate the mean for each group
passCoverage_grouped = passCoverage_analysis.groupby('pff_passCoverage').mean()

# Display the results
print(passCoverage_grouped)
# Plot the bar chart
passCoverage_grouped.plot(kind='bar', figsize=(10, 6))
plt.title('Comparison of Metrics for Different Pass Coverage')
plt.xlabel('Pass Coverage')
plt.ylabel('Mean Value')
plt.xticks(ticks=[0, 1], rotation=0)
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
plays['isShotgun'] = plays['playDescription'].str.contains(r'\(Shotgun\)', case=False, na=False)

# Select relevant columns
shotgun_analysis = plays[['isShotgun','yardsGained', 'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'expectedPointsAdded']]


# Select relevant columns for normalization
columns_to_normalize = ['yardsGained', 'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'expectedPointsAdded']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize the selected columns
shotgun_analysis[columns_to_normalize] = scaler.fit_transform(shotgun_analysis[columns_to_normalize])

# Group by isShotgun and calculate the mean for each group
shotgun_grouped_normalized = shotgun_analysis.groupby('isShotgun').mean()

# Display the results
print(shotgun_grouped_normalized)

# Plot the bar chart
ax = shotgun_grouped_normalized.plot(kind='bar', figsize=(10, 6))
plt.title('Comparison of Normalized Metrics for Shotgun and Non-Shotgun Plays')
plt.xlabel('Shotgun')
plt.ylabel('Mean Normalized Value')
plt.xticks(ticks=[0, 1], labels=['Not Shotgun', 'Shotgun'], rotation=0)
plt.legend(loc='best')

# Add values on top of each column
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.2f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'center', 
                xytext = (0, 9), 
                textcoords = 'offset points')

plt.show()

In [ ]:
# Group by quarter and isShotgun, then count the number of plays
shotgun_count_by_quarter = plays.groupby(['quarter', 'isShotgun'])['playId'].transform('count')

# Display the first few rows to verify the new column
shotgun_count_by_quarter.head()

In [ ]:
# Check if playDescription contains "(Shotgun)"
plays['isShotgun'] = plays['playDescription'].str.contains(r'\(Shotgun\)', case=False, na=False)

# Count the occurrences
shotgun_counts = plays['isShotgun'].value_counts()

# Plot the bar chart
plt.figure(figsize=(8, 6))
shotgun_counts.plot(kind='bar', color=['blue', 'orange'])
plt.xticks(ticks=[0, 1], labels=['Not Shotgun', 'Shotgun'], rotation=0)
plt.xlabel('Play Type')
plt.ylabel('Count')
plt.title('Count of Plays with and without (Shotgun)')
plt.show()

In [ ]:
runConceptSecondary=plays['pff_runConceptSecondary']
#runConceptSecondary.value_counts().plot(kind='bar')
plays['playDescription'].tail()

In [ ]:
plays['isDoubleCloud'] = plays['pff_passCoverage'].str.contains('Cover-3 Double Cloud', case=False, na=False)
plays['isDoubleCloud'].value_counts()

In [ ]:
freq_table=plays['pff_passCoverage'].value_counts()
print(freq_table)

plt.figure(figsize=(10, 6))
plt.bar(freq_table.index, freq_table.values, color='skyblue')
plt.xlabel('Pass Coverage')
plt.ylabel('Count')
plt.title('Frequency of Pass Coverage')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Select relevant columns
passCoverage_analysis = plays[['pff_passCoverage','yardsGained', 'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'expectedPointsAdded']]

# Group by isShotgun and calculate the mean for each group
passCoverage_grouped = passCoverage_analysis.groupby('pff_passCoverage').mean()

# Display the results
print(passCoverage_grouped)

# Line chart visualization
plt.figure(figsize=(14, 8))

# Plot each metric as a separate line
for column in passCoverage_grouped.columns:
    plt.plot(passCoverage_grouped.index, passCoverage_grouped[column], marker='o', label=column)

# Enhance the chart
plt.title("Metrics Comparison Across Pass Coverage Types", fontsize=16)
plt.xlabel("Pass Coverage Type", fontsize=12)
plt.ylabel("Mean Value", fontsize=12)
plt.xticks(rotation=45)
plt.legend(title="Metrics", loc='upper left')
plt.grid(True)

# Show the chart
plt.tight_layout()
plt.show()


In [ ]:
player_play[player_play['hadRushAttempt']==1].groupby(['teamAbbr']).rushingYards.mean().sort_values(ascending=False)[:5]
player_play[player_play['hadRushAttempt']==1].groupby(['teamAbbr']).rushingYards.mean().sort_values(ascending=False)[:5].plot(kind='bar')

In [ ]:
player_play[player_play['hadRushAttempt']==1].groupby(['teamAbbr']).rushingYards.mean().sort_values(ascending=True)[:5].plot(kind='bar')

In [ ]:
player_play[player_play['hadPassReception']==1].groupby(['teamAbbr']).receivingYards.mean().sort_values(ascending=False)[:5]

In [ ]:
player_play[player_play['hadPassReception']==1].groupby(['teamAbbr']).receivingYards.mean().sort_values(ascending=False)[:5].plot(kind='bar')

In [ ]:
avg_rush=player_play[player_play['hadRushAttempt']==1].groupby(['nflId']).rushingYards.mean().sort_values(ascending=False).rename('avg_rush_yards')
num_rush=player_play[player_play['hadRushAttempt']==1].groupby(['nflId']).rushingYards.count().sort_values(ascending=False).rename('num_rush')

In [ ]:
rush_stat=pd.concat([avg_rush,num_rush],axis=1).rename_axis('nflId').reset_index()
rusher_stat=pd.merge(left=rush_stat,right=players[['nflId','displayName']],how='inner',on='nflId')
##get only Id and name from players
play_info=plays[['gameId','playId','offenseFormation']].groupby(['gameId','playId','offenseFormation']).count().reset_index(level= ['gameId','playId','offenseFormation'])

rush_stat[rush_stat['num_rush']>10].sort_values(by='avg_rush_yards',ascending=False)

In [ ]:
rusher_stat[rusher_stat['num_rush']>10].sort_values(by='avg_rush_yards',ascending=False)[:5]

In [47]:
player_rush_yards=player_play[player_play['hadRushAttempt']==1][['nflId','gameId','playId','rushingYards','teamAbbr']].groupby(['nflId','gameId','playId','teamAbbr']).sum().reset_index()
player_rush_att=player_play[player_play['hadRushAttempt']==1][['nflId','gameId','playId']].groupby(['nflId','gameId']).mean().reset_index()
player_rush_stats=pd.merge(left=player_rush_yards,right=player_rush_att,how='inner',on=['nflId','gameId','playId'])
rush_stat=pd.concat([avg_rush,num_rush],axis=1).rename_axis('nflId').reset_index()
rusher_stat=pd.merge(left=player_rush_yards,right=players[['nflId','displayName']],how='inner',on='nflId')
##get only Id and name from players
play_info=plays[['gameId','playId','offenseFormation','receiverAlignment','passResult']].groupby(['gameId','playId','offenseFormation','receiverAlignment','passResult']).count().reset_index(level= ['gameId','playId','offenseFormation','receiverAlignment','passResult'])
player_rec_yards=player_play[player_play['hadPassReception']==1][['nflId','gameId','playId','receivingYards','teamAbbr']].groupby(['nflId','gameId','playId','teamAbbr']).sum().reset_index()
rec_stat=pd.merge(left=player_rec_yards,right=players[['nflId','displayName']],how='inner',on='nflId')
rec_data=pd.merge(left=rec_stat,right=play_info,how='inner',on=['playId','gameId'])
rush_data=pd.merge(left=rusher_stat,right=play_info,how='inner',on=['playId','gameId'])
yard_data=pd.concat([rush_data,rec_data],axis=0)

C:\Users\lanvu\AppData\Local\Temp\ipykernel_7548\3932252316.py:3: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  player_rush_stats=pd.merge(left=player_rush_yards,right=player_rush_att,how='inner',on=['nflId','gameId','playId'])


In [38]:
player_rec_yards=player_play[player_play['hadPassReception']==1][['nflId','gameId','playId','receivingYards','teamAbbr']].groupby(['nflId','gameId','playId','teamAbbr']).sum().reset_index()
rec_stat=pd.merge(left=player_rec_yards,right=players[['nflId','displayName']],how='inner',on='nflId')
rec_data=pd.merge(left=rec_stat,right=play_info,how='inner',on=['playId','gameId'])

In [ ]:
rec_data[['offenseFormation','receivingYards']].groupby('offenseFormation').mean()

,receivingYards
offenseFormation,
EMPTY,10.341270
I_FORM,12.094675
JUMBO,3.533333
PISTOL,11.568182
SHOTGUN,10.730003
SINGLEBACK,12.289744
WILDCAT,9.900000


In [ ]:
rec_data[['offenseFormation','receivingYards']].groupby('offenseFormation').count()

In [ ]:
rush_data[['teamAbbr','rushingYards','offenseFormation']].groupby(['teamAbbr','offenseFormation']).mean()

In [ ]:
rush_data[['offenseFormation','rushingYards']].groupby(['offenseFormation']).mean().sort_values(by='rushingYards',ascending=False)

In [ ]:
rush_data[['offenseFormation','rushingYards']].groupby(['offenseFormation']).count()

In [ ]:
yard_data=pd.concat([rush_data,rec_data],axis=0)

In [ ]:
yard_data[yard_data['teamAbbr']=='ARI'][['offenseFormation','rushingYards','receivingYards']].groupby('offenseFormation').mean()

In [ ]:
yard_data[yard_data['teamAbbr']=='ARI'][['gameId','offenseFormation','rushingYards','receivingYards']].groupby(['gameId','offenseFormation']).count()

In [ ]:
yard_data[yard_data['teamAbbr']=='TB'][['displayName','offenseFormation','receivingYards']].groupby(['displayName','offenseFormation']).mean()

In [48]:
yard_data

,nflId,gameId,playId,teamAbbr,rushingYards,displayName,offenseFormation,receiverAlignment,passResult,receivingYards
0,25511,2022102301,2725,TB,1.0,Tom Brady,SHOTGUN,2x2,R,NaN
1,25511,2022102700,3083,TB,1.0,Tom Brady,SHOTGUN,2x2,R,NaN
2,29851,2022091812,663,GB,7.0,Aaron Rodgers,SHOTGUN,2x2,R,NaN
3,29851,2022100211,1704,GB,1.0,Aaron Rodgers,SHOTGUN,2x2,R,NaN
4,29851,2022103012,3161,GB,11.0,Aaron Rodgers,EMPTY,3x2,R,NaN
...,...,...,...,...,...,...,...,...,...,...
5620,55157,2022102303,1264,DAL,NaN,Peyton Hendershot,SINGLEBACK,3x1,C,3.0
5621,55157,2022102303,3435,DAL,NaN,Peyton Hendershot,SINGLEBACK,2x2,C,2.0
5622,55157,2022103002,491,DAL,NaN,Peyton Hendershot,SHOTGUN,3x1,C,11.0
5623,55157,2022103002,544,DAL,NaN,Peyton Hendershot,SHOTGUN,3x1,C,4.0


In [58]:
yard_data['rush'] = yard_data['rushingYards'].apply(lambda x: 0 if pd.isna(x) else 1)

In [59]:
yard_data

,nflId,gameId,playId,teamAbbr,rushingYards,displayName,offenseFormation,receiverAlignment,passResult,receivingYards,rush
0,25511,2022102301,2725,TB,1.0,Tom Brady,SHOTGUN,2x2,R,NaN,1
1,25511,2022102700,3083,TB,1.0,Tom Brady,SHOTGUN,2x2,R,NaN,1
2,29851,2022091812,663,GB,7.0,Aaron Rodgers,SHOTGUN,2x2,R,NaN,1
3,29851,2022100211,1704,GB,1.0,Aaron Rodgers,SHOTGUN,2x2,R,NaN,1
4,29851,2022103012,3161,GB,11.0,Aaron Rodgers,EMPTY,3x2,R,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
5620,55157,2022102303,1264,DAL,NaN,Peyton Hendershot,SINGLEBACK,3x1,C,3.0,0
5621,55157,2022102303,3435,DAL,NaN,Peyton Hendershot,SINGLEBACK,2x2,C,2.0,0
5622,55157,2022103002,491,DAL,NaN,Peyton Hendershot,SHOTGUN,3x1,C,11.0,0
5623,55157,2022103002,544,DAL,NaN,Peyton Hendershot,SHOTGUN,3x1,C,4.0,0


In [55]:
##Basic logistics
catagorical_var =['offenseFormation','teamAbbr','passResult','receiverAlignment']
target_var = 'rush'

x=yard_data[catagorical_var]
y=yard_data[target_var]

In [60]:
yard_data['rush'].value_counts()

0    5625
1     400
Name: rush, dtype: int64